# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://www.shop-apotheke.com/")
ed.links

['/nx/newsletter/',
 '/online-arzt-service.htm',
 '/lp/gutscheine/',
 '/premium-lieferung/',
 '/faq.htm',
 '/versandapotheke_kontakt.htm',
 '/about.htm',
 '/',
 '/nx/login/',
 '/e-rezept-scannen/',
 '/nx/cart/',
 '/e-rezept-einloesen/',
 '/lp/eigenmarken/',
 '/angebote/',
 '/gratis-aktionen/',
 '/premium-beauty/',
 '/fitness-sportequipment/',
 '/homoeopathisch-beruhigung-schlaf-nerven/',
 '/kontaktlinsen/',
 '/redpoints-promotions/',
 '/now/',
 '/online-arzt-service.htm',
 '/versandapotheke_kontakt.htm',
 '/arzneimittel/',
 '/baby-und-familie/',
 '/beautyprodukte-und-pflegeprodukte/',
 '/bio-naturprodukte/',
 '/buecher/',
 '/ernaehrung-sport-abnehmen/',
 '/homoeopathie-naturprodukte/',
 '/hygiene-haushalt/',
 '/nachhaltigkeit/',
 '/sanitaetshaus/',
 '/sport-fitness/',
 '/tierapotheke/',
 '/markenshops/',
 '/ratgeber/',
 '/medikamenten-lexikon/',
 '/e-rezept-einloesen/',
 '/magazin/familie/',
 '/nx/redpoints/',
 '/lp/gutscheine/',
 '/medikamenten-app/',
 '/lp/shop-apotheke-app/',
 '/wec

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage, that's an online pharmacy in Germany. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages. Beware that some of the links might have german words.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage, that's an online pharmacy in Germany. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages. Beware that some of the links might have german words.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://www.shop-apotheke.com/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/nx/newsletter/
/online-arzt-service.htm
/lp/gutscheine/
/premium-lieferung/
/faq.htm
/versandapotheke_kontakt.htm
/about.htm
/
/nx/login/
/e-rezept-scannen/
/nx/cart/
/e-rezept-einloesen/
/lp/eigenmarken/
/angebote/
/gratis-aktionen/
/premium-beauty/
/fitness-sportequipment/
/homoeopathisch-beruhigung-schlaf-nerven/
/kontaktlinsen/
/redpoints-promotions/
/now/
/online-arzt-service.htm
/versandapotheke_kontakt.htm
/arzneimittel/
/baby-und-familie/
/beautyprodukte-und-pflegeprodukte/
/bio-naturprodukte/
/buecher/
/ernaehrung-sport-abnehmen/
/homoeopathie-naturprodukte/
/hygiene-haushalt/
/nachhaltigkeit/
/sanitaetshaus/
/sport-fitness/
/tierapotheke/
/markenshops/
/ratgeber/
/medikamenten

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
get_links("https://www.shop-apotheke.com/")

{'links': [{'type': 'about page',
   'url': 'https://www.shop-apotheke.com/about.htm'},
  {'type': 'careers page',
   'url': 'https://www.redcare-pharmacy.com/de/careers'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [11]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [12]:
print(get_all_details("https://www.shop-apotheke.com/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.shop-apotheke.com/about.htm'}, {'type': 'careers page', 'url': 'https://www.redcare-pharmacy.com/de/careers'}, {'type': 'company homepage', 'url': 'https://www.shop-apotheke.com'}]}
Landing page:
Webpage Title:
Online Apotheke für Deutschland ▷ Shop Apotheke
Webpage Contents:
Newsletter - 5€ Gutschein
Online-Arzt
Gutscheine
Unser Now! Service
FAQ
Kontakt
Über uns
Suchen
Anmelden
E-Rezept
0
Warenkorb
Menubar
Kategorien
Wir empfehlen
Rezept einlösen
Unsere Eigenmarken
Top Angebote
Aktionen & Sets
Premium Beauty
Fitnessgeräte
Beruhigung, Schlaf & Nerven
Kontaktlinsen
RedPoints Aktionen
Now! Produkte
Online-Arzt-Service
Kontakt
Unsere Kategorien
Arzneimittel & Gesundheit
Baby & Familie
Beauty & Pflege
Bio & Naturprodukte
Bücher
Ernährung & Abnehmen
Homöopathie
Hygiene & Haushalt
Nachhaltigkeit
Sanitätshaus
Sport & Fitness
Tierapotheke & Tierbedarf
Auch interessant
Markenshops
Shop Apotheke Ratgeber
Medikamenten-Lexikon
Das 

In [13]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website, that's an online pharmacy in Germany \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [14]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [15]:
get_brochure_user_prompt("Shop Apotheke", "https://www.shop-apotheke.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.shop-apotheke.com/about.htm'}, {'type': 'careers page', 'url': 'https://www.redcare-pharmacy.com/de/careers'}, {'type': 'contact page', 'url': 'https://www.shop-apotheke.com/versandapotheke_kontakt.htm'}, {'type': 'home page', 'url': 'https://www.shop-apotheke.com'}]}


'You are looking at a company called: Shop Apotheke\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nOnline Apotheke für Deutschland ▷ Shop Apotheke\nWebpage Contents:\nNewsletter - 5€ Gutschein\nOnline-Arzt\nGutscheine\nUnser Now! Service\nFAQ\nKontakt\nÜber uns\nSuchen\nAnmelden\nE-Rezept\n0\nWarenkorb\nMenubar\nKategorien\nWir empfehlen\nRezept einlösen\nUnsere Eigenmarken\nTop Angebote\nAktionen & Sets\nPremium Beauty\nFitnessgeräte\nBeruhigung, Schlaf & Nerven\nKontaktlinsen\nRedPoints Aktionen\nNow! Produkte\nOnline-Arzt-Service\nKontakt\nUnsere Kategorien\nArzneimittel & Gesundheit\nBaby & Familie\nBeauty & Pflege\nBio & Naturprodukte\nBücher\nErnährung & Abnehmen\nHomöopathie\nHygiene & Haushalt\nNachhaltigkeit\nSanitätshaus\nSport & Fitness\nTierapotheke & Tierbedarf\nAuch interessant\nMarkenshops\nShop Apotheke Ratgeber\nMedikamenten-Lexikon\nDas E-Re

In [16]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [17]:
create_brochure("Shop Apotheke", "https://www.shop-apotheke.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.shop-apotheke.com/about.htm'}, {'type': 'careers page', 'url': 'https://www.redcare-pharmacy.com/de/careers'}, {'type': 'company page', 'url': 'https://www.redcare-pharmacy.com/de/ueber-uns'}]}


```markdown
# Shop Apotheke: Ihre Online Apotheke für Deutschland

## Über uns

With a focus on customer satisfaction, Shop Apotheke positions itself as the leading online pharmacy in Germany. We offer a vast selection of medications, health products, and beauty items, ensuring that our customers can find everything they need from the comfort of their home. Our commitment to affordable pricing, high-quality products, and fast delivery makes us a trusted choice for individuals and families.

## Services & Benefits

- **E-Rezept**: Conveniently redeem your electronic prescriptions with rapid home delivery.
- **Now! Service**: Enjoy same-day delivery for orders placed before a certain time.
- **Diverse Product Range**: From pharmaceuticals and homoeopathic remedies to beauty and wellness products, we supply everything you may need, including essentials for your pets.
- **Customer Loyalty Program**: Earn RedPoints through purchases to enjoy greater savings.
- **Mobile App**: Download our Medikations-App and get up to 10% off on your first E-Rezept order.

## Company Culture

At Shop Apotheke, we foster an innovative and customer-centric culture. Our team is dedicated to providing top-notch service and catering to the healthcare needs of our clients. We believe in maintaining a collaborative atmosphere that encourages personal and professional growth, allowing our members to contribute meaningfully to the community we serve.

## Customers

Our diverse customer base includes individuals seeking affordable medication, families in need of health and wellness products, and pet owners searching for reliable veterinary supplies. We cater to all age groups and demographics, ensuring accessibility and satisfying various health and beauty needs.

## Careers

Join the Shop Apotheke family! We are constantly looking for passionate individuals seeking careers in the health sector. We offer competitive benefits, opportunities for advancement, and a chance to work in a pivotal role within the growing online pharmacy industry. If you're interested in making a difference and being a part of our innovative team, visit our careers page to explore available opportunities.

## Contact Us

If you have any questions, please don’t hesitate to reach out through our **[Kontakt](#)** page for quick responses!

---

**Shop Apotheke** - Where Convenience Meets Healthcare.
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [18]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [19]:
stream_brochure("Shop Apotheke", "https://www.shop-apotheke.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.shop-apotheke.com/about.htm'}, {'type': 'careers page', 'url': 'https://www.redcare-pharmacy.com/de/careers'}, {'type': 'contact page', 'url': 'https://www.shop-apotheke.com/versandapotheke_kontakt.htm'}]}



# Shop Apotheke Brochure

## Welcome to Shop Apotheke
Shop Apotheke is your premier online pharmacy in Germany, offering a wide range of health and wellness products at competitive prices. We are committed to providing a seamless shopping experience with the convenience of home delivery. Whether you need medication, beauty products, or health advice, Shop Apotheke is here to help.

## Our Services
- **E-Rezept**: Easily redeem your electronic prescriptions with quick home delivery.
- **Now! Service**: Same-day delivery options available for urgent needs.
- **RedPoints**: Join our customer loyalty program to save up to 10€ on future purchases.
- **Online-Arzt**: Access consultations with qualified doctors right from your home.
- **Health Resources**: Explore our medication guide and advice on various health conditions.

## Product Categories
We offer an extensive selection of:
- Prescription & over-the-counter medications
- Health and wellness products
- Beauty & personal care items
- Homeopathy and natural products
- Pet medications

## Why Choose Us?
- **Quality Assurance**: We prioritize high-quality products and reliable service, ensuring that your health is in safe hands.
- **Customer-Centric Approach**: Our team is dedicated to addressing your needs, and we strive for excellence in customer service.
- **Expert Guidance**: Access expert health information and articles through our E-Health Magazine and online resources.

## Company Culture
At Shop Apotheke, we believe in fostering a collaborative and inclusive work environment. Our team is passionate about health, innovation, and providing exceptional service. We encourage professional growth and value the diverse perspectives of our employees. Join us as we continue to improve the lives of our customers across Germany.

## Career Opportunities
We are always on the lookout for talented individuals who share our commitment to health and wellness. Explore exciting career opportunities with us, whether you're interested in roles in customer service, logistics, or healthcare. Become a part of a dynamic team that drives innovation in the online pharmacy sector.

## Join Us Today!
Stay updated by subscribing to our newsletter and receive a €5 voucher on your next purchase! Download our app and enjoy exclusive offers, medication reminders, and health insights directly at your fingertips.

For more information, visit our website or contact us through our support channels. Let's take a step towards better health together!

### Contact Us
- **Website**: [Shop Apotheke](https://www.shop-apotheke.com)
- **Email**: support@shop-apotheke.com
- **Phone**: 0800-1234567

---

Thank you for choosing Shop Apotheke, where your health is our priority!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>